<a href="https://colab.research.google.com/github/debiprasad2004/ASSESSMENT-RECOMMENDATION-TOOL/blob/main/assessmetrecommendationtool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Hugging Face Transformers & Dependencies
!pip install -q transformers accelerate peft bitsandbytes datasets sentencepiece evaluate


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

prompt = "Generate a multiple-choice programming question about Python functions:\n"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Generate a multiple-choice programming question about Python functions:

Question: What type of function is `def print_hello():` in Python?

Answers:
1. Return value
2. Arguments
3. Function object
4. None

Explanation:
A function is a block of code that takes a set of input parameters and returns a value. Functions can be defined using the `def` keyword, which specifies the function name and the parameters. The function body contains code that performs a task


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Define question and answer
question = "Which method is used to add an item to a list in Python?"
correct_answer = "append()"
prompt = f"Generate three distractors for the correct answer '{correct_answer}' in the question: '{question}'"

# Encode input
inputs = t5_tokenizer(prompt, return_tensors="pt")

# Generate
outputs = t5_model.generate(**inputs, max_new_tokens=50)
distractors = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated distractors:", distractors)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generated distractors: True


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

reference = ["Which method is used to add an item to a list in Python?"]
prediction = ["Which function is used to add a new item to a list in Python?"]

score = bleu.compute(predictions=prediction, references=[reference])
print("BLEU score:", score)


BLEU score: {'bleu': 0.5828233954152654, 'precisions': [0.8, 0.6428571428571429, 0.5384615384615384, 0.4166666666666667], 'brevity_penalty': 1.0, 'length_ratio': 1.0714285714285714, 'translation_length': 15, 'reference_length': 14}


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Example: Download a dataset
!kaggle datasets download -d shreeharis754/cse-mcq-dataset

# Unzip the dataset
!unzip -q cse-mcq-dataset.zip -d cse-mcq-dataset

Dataset URL: https://www.kaggle.com/datasets/shreeharis754/cse-mcq-dataset
License(s): CC0-1.0
cse-mcq-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace cse-mcq-dataset/CSE_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: o
error:  invalid response [o]
replace cse-mcq-dataset/CSE_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
df = pd.read_csv("cse-mcq-dataset/CSE_Dataset.csv")

In [ ]:
print(df.columns.tolist())


['Question', 'Option 1', 'Option 2', 'Option 3', 'Option 4', 'Answer']


In [ ]:
import json

dataset_for_sft = []

for _, row in df.iterrows():
    prompt = "Generate a computer science multiple-choice question:\n"
    question = row["Question"]
    options = [row["Option 1"], row["Option 2"], row["Option 3"], row["Option 4"]]
    completion = f"{question}\nA) {options[0]}\nB) {options[1]}\nC) {options[2]}\nD) {options[3]}\nAnswer: "

    # Find correct label (A, B, C, D)
    try:
        correct_index = options.index(row["Answer"])
        correct_label = ["A", "B", "C", "D"][correct_index]
    except ValueError:
        correct_label = "Unknown"

    completion += correct_label
    dataset_for_sft.append({"prompt": prompt, "completion": completion})

# Save to JSON
with open("train_data_sft.json", "w") as f:
    json.dump(dataset_for_sft, f, indent=2)

print("✅ Dataset saved as train_data_sft.json")


✅ Dataset saved as train_data_sft.json


In [ ]:
pip install transformers datasets peft accelerate evaluate matplotlib


In [ ]:
from datasets import Dataset
import json

# Load the JSON data
with open("train_data_sft.json") as f:
    data = json.load(f)

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(data)

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Tokenization function
def tokenize(example):
    full_text = example["prompt"] + example["completion"]
    tokens = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Apply to dataset
tokenized_dataset = dataset.map(tokenize, remove_columns=["prompt", "completion"])


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
pip install --upgrade transformers
